We will use historical data to build a predictive model that forecasts the status of applications or requests.

In [1]:
# Install necessary libraries
!pip install pandas numpy scikit-learn flask-ngrok

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok

# Sample data: Historical application statuses
data = {
    'application_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'applicant_age': [25, 34, 22, 45, 33, 27, 29, 40, 30, 35],
    'application_type': [1, 2, 1, 1, 2, 1, 2, 2, 1, 2],
    'previous_applications': [0, 1, 0, 2, 1, 0, 1, 2, 0, 1],
    'status': [1, 0, 1, 0, 1, 1, 0, 0, 1, 1]  # 1: Approved, 0: Denied
}

df = pd.DataFrame(data)

# Feature engineering
X = df[['applicant_age', 'application_type', 'previous_applications']]
y = df['status']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Predict and evaluate the model
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Save the model
import joblib
joblib.dump(clf, 'predictive_model.pkl')

# Flask app for the self-service portal
app = Flask(__name__)
run_with_ngrok(app)

@app.route('/predict_status', methods=['POST'])
def predict_status():
    data = request.get_json()
    features = [data['applicant_age'], data['application_type'], data['previous_applications']]
    model = joblib.load('predictive_model.pkl')
    prediction = model.predict([features])
    status = 'Approved' if prediction[0] == 1 else 'Denied'
    return jsonify({'status': status})

app.run()


Accuracy: 0.50
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-10:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 791, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urll